In [2]:
import pathlib

path=str(pathlib.Path().resolve())+"/assets/"

In [3]:
texts=[]
with open(path+'funes.txt','r') as file:
    texts = file.read().splitlines() 
texts

['\ufeffFunes el memorioso',
 'Jorge Luis Borges\xa0',
 '',
 '\tLo recuerdo (yo no tengo derecho a pronunciar ese verbo sagrado, sólo un hombre en la tierra tuvo derecho y ese hombre ha muerto) con una oscura pasionaria en la mano, viéndola como nadie la ha visto, aunque la mirara desde el crepúsculo del día hasta el de la noche, toda una vida entera. Lo recuerdo, la cara taciturna y aindiada y singularmente remota, detrás del cigarrillo. Recuerdo (creo) sus manos afiladas de trenzado. Recuerdo cerca de esas manos un mate, con las armas de la Banda Oriental; recuerdo en la ventana de la casa una estera amarilla, con un vago paisaje lacustre. Recuerdo claramente su voz; la voz pausada, resentida y nasal del orillero antiguo, sin los silbidos italianos de ahora. Más de tres veces no lo vi; la última, en 1887... Me parece muy feliz el proyecto de que todos aquellos que lo trataron escriban sobre él; mi testimonio será acaso el más breve y sin duda el más pobre, pero no el menos imparcial 

In [4]:
texts=list(filter(lambda x: x!="",texts))

In [5]:
texts

['\ufeffFunes el memorioso',
 'Jorge Luis Borges\xa0',
 '\tLo recuerdo (yo no tengo derecho a pronunciar ese verbo sagrado, sólo un hombre en la tierra tuvo derecho y ese hombre ha muerto) con una oscura pasionaria en la mano, viéndola como nadie la ha visto, aunque la mirara desde el crepúsculo del día hasta el de la noche, toda una vida entera. Lo recuerdo, la cara taciturna y aindiada y singularmente remota, detrás del cigarrillo. Recuerdo (creo) sus manos afiladas de trenzado. Recuerdo cerca de esas manos un mate, con las armas de la Banda Oriental; recuerdo en la ventana de la casa una estera amarilla, con un vago paisaje lacustre. Recuerdo claramente su voz; la voz pausada, resentida y nasal del orillero antiguo, sin los silbidos italianos de ahora. Más de tres veces no lo vi; la última, en 1887... Me parece muy feliz el proyecto de que todos aquellos que lo trataron escriban sobre él; mi testimonio será acaso el más breve y sin duda el más pobre, pero no el menos imparcial del v

In [6]:
#!pip install -U sentence-transformers


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
embeddings = model.encode(texts)
print(embeddings)

c:\Users\diego.guizanlopez\AppData\Local\miniconda3\envs\ia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[ 1.1161914  -0.5944319  -0.16006914 ...  0.04201844  0.17456518
  -0.13278331]
 [ 0.7258058  -0.2930283   0.1573473  ... -0.53282744 -1.0715344
  -0.3209751 ]
 [-0.24173345 -0.39351264 -0.00498043 ... -0.386119   -0.37455675
  -0.1661987 ]
 ...
 [-0.27665073  0.38812798  0.12186419 ... -0.56073546 -0.4460481
   0.10735115]
 [ 1.0342375   0.01015015 -0.1277491  ... -0.08763277 -0.02247056
   0.08189774]
 [-0.69099593 -0.351554   -0.6079379  ...  0.791455    0.26335335
  -0.21762171]]


In [19]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def tokenizer_text(list):
    tokenizer = AutoTokenizer.from_pretrained('hiiamsid/sentence_similarity_spanish_es')
    model = AutoModel.from_pretrained('hiiamsid/sentence_similarity_spanish_es')

    # Tokenize sentences
    encoded_input = tokenizer(list, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    return mean_pooling(model_output, encoded_input['attention_mask'])


In [20]:
sentence_embeddings = tokenizer_text(texts)

In [14]:
sentence_embeddings

17

In [27]:
TextBuscar="Muerte de Ireneo Funes"

In [28]:
def calculate_similarity(embeddings_1, embeddings_2):
    # Calcula la similitud del coseno entre dos conjuntos de embeddings
    return torch.nn.functional.cosine_similarity(embeddings_1, embeddings_2)

# Entrena el modelo con los párrafos originales
original_embeddings = tokenizer_text(texts)

# Añade un nuevo texto
new_text_embedding = tokenizer_text([TextBuscar])

# Calcula la similitud entre el nuevo texto y los párrafos originales
similarities = calculate_similarity(new_text_embedding, original_embeddings)

# Encuentra el párrafo más similar
most_similar_index = torch.argmax(similarities)
most_similar_paragraph = texts[most_similar_index]


In [ ]:
most_similar_paragraph

'\tIreneo Funes murió en 1889, de una congestión pulmonar.'